In [1]:
import os

In [2]:
%pwd

'c:\\Users\\LENOVO\\Desktop\\Projects\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\LENOVO\\Desktop\\Projects\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    loclal_data_file: Path
    unzip_dir: Path

In [7]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
            
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            loclal_data_file=Path(config.loclal_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config


In [10]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [11]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.loclal_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.loclal_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.loclal_data_file))}")

    def extract_zip_file(self):
        '''
        zip_file_path: str
        Extract the zip file into the data directory
        Function return None
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.loclal_data_file, 'r') as   zip_ref:
            zip_ref.extractall(unzip_path)

In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-04 21:20:03,314: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-10-04 21:20:03,457: INFO: common: YAML file: params.yaml loaded successfully]
[2025-10-04 21:20:03,469: INFO: common: Directory created at: artifacts]
[2025-10-04 21:20:03,472: INFO: common: Directory created at: artifacts/data_ingestion]
[2025-10-04 21:20:09,399: INFO: 3145716133: artifacts\data_ingestion\samsum_data.zip downloaded! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4F14:3E1138:30B440:73020A:68E14226
Accept-Ranges: bytes
Date: Sat, 04 Oct 2025 15:50:07 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm721005